In [1]:
pip install torch pymongo


                                              0.0/172.3 MB ? eta -:--:--
                                              0.2/172.3 MB 9.6 MB/s eta 0:00:19
                                              0.5/172.3 MB 6.7 MB/s eta 0:00:26
                                              0.9/172.3 MB 7.5 MB/s eta 0:00:23
                                              1.5/172.3 MB 8.9 MB/s eta 0:00:20
                                              2.1/172.3 MB 9.5 MB/s eta 0:00:18
                                              2.6/172.3 MB 9.9 MB/s eta 0:00:18
                                             3.2/172.3 MB 10.2 MB/s eta 0:00:17
                                              3.6/172.3 MB 9.9 MB/s eta 0:00:18
                                              4.0/172.3 MB 9.9 MB/s eta 0:00:17
     -                                       4.5/172.3 MB 10.2 MB/s eta 0:00:17
     -                                        4.7/172.3 MB 9.3 MB/s eta 0:00:19
     -                                        

In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from pymongo import MongoClient
from sklearn.model_selection import train_test_split

In [14]:
import csv

movie_ratings = []

csv_file = 'C:\\Users\\Aayush\\Documents\\Untitled Folder\\ml-latest-small\\ratings.csv'

with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    
    next(csv_reader, None)
    
    for row in csv_reader:
        user_id = int(row[0])   
        movie_id = int(row[1])   
        rating = float(row[2])    
        
        rating_entry = {
            "user_id": user_id,
            "movie_id": movie_id,
            "rating": rating
        }
        
        movie_ratings.append(rating_entry)


In [129]:
import csv

movies = []

csv_file = 'C:\\Users\\Aayush\\Documents\\Untitled Folder\\ml-latest-small\\movies.csv'

with open(csv_file, 'r',  encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    
    next(csv_reader, None)
    
    for row in csv_reader:
        movie_id = int(row[0])       
        movie_name = row[1]
        
        name_entry = {
            "movie_id": movie_id,
            "movie_name": movie_name,
        }
        
        movies.append(name_entry)


In [144]:
# Define a simple user-item matrix
num_users = max(rating['user_id'] for rating in movie_ratings)
num_movies = max(rating['movie_id'] for rating in movie_ratings)
user_item_matrix = np.zeros((num_users, num_movies), dtype=np.float32)

In [17]:
train_indices, test_indices = train_test_split(range(len(movie_ratings)), test_size=0.2, random_state=42)
train_ratings = [movie_ratings[i] for i in train_indices]
test_ratings = [movie_ratings[i] for i in test_indices]

In [145]:
#Populate the matrix with user ratings
for rating in train_ratings:
    user_id = rating['user_id'] - 1  
    movie_id = rating['movie_id'] - 1
    user_item_matrix[user_id, movie_id] = rating['rating']

In [19]:
train_ratings

[{'user_id': 509, 'movie_id': 7347, 'rating': 3.0},
 {'user_id': 326, 'movie_id': 71462, 'rating': 4.0},
 {'user_id': 57, 'movie_id': 2115, 'rating': 3.0},
 {'user_id': 610, 'movie_id': 1127, 'rating': 4.0},
 {'user_id': 462, 'movie_id': 2409, 'rating': 2.0},
 {'user_id': 600, 'movie_id': 2084, 'rating': 3.5},
 {'user_id': 89, 'movie_id': 121342, 'rating': 3.5},
 {'user_id': 343, 'movie_id': 2329, 'rating': 4.0},
 {'user_id': 414, 'movie_id': 36517, 'rating': 4.0},
 {'user_id': 156, 'movie_id': 3072, 'rating': 4.0},
 {'user_id': 305, 'movie_id': 1625, 'rating': 2.5},
 {'user_id': 448, 'movie_id': 440, 'rating': 3.0},
 {'user_id': 57, 'movie_id': 594, 'rating': 4.0},
 {'user_id': 20, 'movie_id': 6157, 'rating': 2.0},
 {'user_id': 313, 'movie_id': 541, 'rating': 5.0},
 {'user_id': 600, 'movie_id': 5064, 'rating': 1.5},
 {'user_id': 304, 'movie_id': 1210, 'rating': 5.0},
 {'user_id': 469, 'movie_id': 1104, 'rating': 5.0},
 {'user_id': 138, 'movie_id': 2707, 'rating': 0.5},
 {'user_id': 21

In [146]:
user_item_tensor = torch.tensor(user_item_matrix)

In [147]:
import torch.optim as optim

In [148]:
class NMFModel(nn.Module):
    def __init__(self, num_users, num_movies, embedding_dim):
        super(NMFModel, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.movie_embeddings = nn.Embedding(num_movies, embedding_dim)
        
    def forward(self, user_indices, movie_indices):
        user_embeds = self.user_embeddings(user_indices)
        movie_embeds = self.movie_embeddings(movie_indices)
        
        user_embeds = torch.relu(user_embeds) 
        movie_embeds = torch.relu(movie_embeds) 
        
        return torch.sum(user_embeds * movie_embeds, dim=1)

In [149]:
embedding_dim = 20
learning_rate = 0.001
num_epochs = 100

In [150]:
model = NMFModel(num_users, num_movies, embedding_dim)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [151]:
for epoch in range(num_epochs):
    optimizer.zero_grad()
    user_indices, movie_indices = torch.where(user_item_tensor != 0)
    ratings = user_item_tensor[user_indices, movie_indices]
    predicted_ratings = model(user_indices, movie_indices)
    
    predicted_ratings = torch.relu(predicted_ratings)
    
    loss = nn.MSELoss()(predicted_ratings, ratings)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/100], Loss: 5.7742
Epoch [2/100], Loss: 5.7741
Epoch [3/100], Loss: 5.7740
Epoch [4/100], Loss: 5.7739
Epoch [5/100], Loss: 5.7738
Epoch [6/100], Loss: 5.7737
Epoch [7/100], Loss: 5.7736
Epoch [8/100], Loss: 5.7735
Epoch [9/100], Loss: 5.7734
Epoch [10/100], Loss: 5.7733
Epoch [11/100], Loss: 5.7732
Epoch [12/100], Loss: 5.7731
Epoch [13/100], Loss: 5.7730
Epoch [14/100], Loss: 5.7729
Epoch [15/100], Loss: 5.7728
Epoch [16/100], Loss: 5.7727
Epoch [17/100], Loss: 5.7726
Epoch [18/100], Loss: 5.7725
Epoch [19/100], Loss: 5.7724
Epoch [20/100], Loss: 5.7723
Epoch [21/100], Loss: 5.7721
Epoch [22/100], Loss: 5.7720
Epoch [23/100], Loss: 5.7719
Epoch [24/100], Loss: 5.7718
Epoch [25/100], Loss: 5.7717
Epoch [26/100], Loss: 5.7716
Epoch [27/100], Loss: 5.7715
Epoch [28/100], Loss: 5.7714
Epoch [29/100], Loss: 5.7713
Epoch [30/100], Loss: 5.7712
Epoch [31/100], Loss: 5.7711
Epoch [32/100], Loss: 5.7710
Epoch [33/100], Loss: 5.7709
Epoch [34/100], Loss: 5.7708
Epoch [35/100], Loss: 5

In [152]:
model.eval()
with torch.no_grad():
    test_user_indices = torch.tensor([rating['user_id'] - 1 for rating in test_ratings])
    test_movie_indices = torch.tensor([rating['movie_id'] - 1 for rating in test_ratings])
    true_ratings = torch.tensor([rating['rating'] for rating in test_ratings], dtype=torch.float32)
    predicted_ratings = model(test_user_indices, test_movie_indices)

In [153]:
mae = nn.L1Loss()(predicted_ratings, true_ratings).item()
rmse = torch.sqrt(nn.MSELoss()(predicted_ratings, true_ratings)).item()
print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}")

MAE: 1.9198, RMSE: 2.3869


In [154]:
user_id = 612  # Choose a user
user_idx = user_id - 1
movie_indices = torch.tensor(range(num_movies))
user_indices = torch.tensor([user_idx] * num_movies)
predicted_ratings = model(user_indices, movie_indices)
sorted_indices = torch.argsort(predicted_ratings, descending=True)

In [166]:
seen_movie_ids = [rating['movie_id'] for rating in movie_ratings if rating['user_id'] == user_id]
unseen_movie_indices = [idx for idx in sorted_indices if idx.item() + 1 not in seen_movie_ids]

top_unseen_movie_ids = [idx.item() + 1 for idx in unseen_movie_indices[:10]]
print(f"Top recommended unseen movies for user {user_id}: {top_unseen_movie_ids}")

Top recommended unseen movies for user 612: [44545, 184936, 180525, 96407, 191324, 44887, 36321, 192253, 17470, 167556]


In [142]:
print("Please add ratings for the movies you've seen.")

max_user_id = max(rating['user_id'] for rating in movie_ratings)

new_user_id = max_user_id + 1
new_user_ratings = [0.0] * num_movies
while True:
    movie_id = int(input("Enter the movie ID (1 to {}), or 0 to finish: ".format(num_movies)))
    
    if movie_id == 0:
        break
    
    if movie_id not in [movie["movie_id"] for movie in movies]:
        print("Invalid movie ID. Please enter a valid movie ID.")
        continue
    
    while True:
        try:
            rating = float(input("Enter your rating for Movie {}: ".format(movie_id)))
            if 0 < rating <= 5:
                break
            else:
                print("Rating must be between 0 and 5 (inclusive). Please enter a valid rating.")
        except ValueError:
            print("Invalid input. Please enter a valid rating.")
    
    rounded_rating = round(rating, 1)
    
    rating_entry = {
        "user_id": new_user_id,
        "movie_id": movie_id,
        "rating": rating
    }
    
    train_ratings.append(rating_entry)
    movie_ratings.append(rating_entry)
    
    new_user_ratings[movie_id - 1] = rating
    
    more_ratings = input("Do you want to add another rating? (yes/no): ")
    if more_ratings.lower() != 'yes':
        break
        
print("Your user id is", new_user_id)
print(new_user_ratings)

Please add ratings for the movies you've seen.
Enter the movie ID (1 to 193609), or 0 to finish: 34
Enter your rating for Movie 34: 2
Do you want to add another rating? (yes/no): no
Your user id is 612
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.